# Imports \ Device \ Seed

In [1]:
######################### Imports ####################################
import io
import os
import math
import pickle
import zipfile
import torch
import numpy as np
import pandas as pd
import random as random
import matplotlib.pyplot as plt
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.distributions import Categorical
from tabulate import tabulate
from random import shuffle
from copy import deepcopy
from pathlib import Path
from urllib.error import URLError
from urllib.request import urlopen
########################## Device #####################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(0)
########################### Seed ######################################
RND_SEED = 123
SEED99 = 123
random.seed(RND_SEED)
torch.manual_seed(RND_SEED)
RANDOM_STATE = RND_SEED
np.random.seed(RND_SEED)
########################### Display ####################################
from IPython.core.display import display, HTML
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})
display(HTML("<style>.container { width:95% !important; }</style>"))

# Data Processing

In [2]:
def try_download(url, download_path):
    archive_name = url.split('/')[-1]
    folder_name, _ = os.path.splitext(archive_name)

    try:
        r = urlopen(url)
    except URLError as e:
        print('Cannot download the data. Error: %s' % s)
        return

    assert r.status == 200
    data = r.read()

    with zipfile.ZipFile(io.BytesIO(data)) as arch:
        arch.extractall(download_path)

    print('The archive is extracted into folder: %s' % download_path)
    
#################################################################################

def read_data(path):
    files = {}
    for filename in path.glob('*'):
        if filename.suffix == '.csv':
            files[filename.stem] = pd.read_csv(filename)
        elif filename.suffix == '.dat':
            if filename.stem == 'ratings':
                columns = ['userId', 'movieId', 'rating', 'timestamp']
            else:
                columns = ['movieId', 'title', 'genres']
            data = pd.read_csv(filename, sep='::', names=columns, engine='python')
            files[filename.stem] = data
    return files['ratings'], files['movies']

#################################################################################

def create_dataset(ratings, top=None):
    if top is not None:
        ratings.groupby('userId')['rating'].count()

    unique_users = ratings.userId.unique()
    user_to_index = {old: new for new, old in enumerate(unique_users)}
    new_users = ratings.userId.map(user_to_index)

    unique_movies = ratings.movieId.unique()
    movie_to_index = {old: new for new, old in enumerate(unique_movies)}
    new_movies = ratings.movieId.map(movie_to_index)

    index_to_movieId = dict(map(reversed, movie_to_index.items()))
    index_to_userId = dict(map(reversed, user_to_index.items()))

    n_users = unique_users.shape[0]
    n_movies = unique_movies.shape[0]

    X = pd.DataFrame({'userId': new_users, 'movieId': new_movies})
    y = ratings['rating'].astype(np.float32)
    t = ratings['timestamp'].astype(np.float64)

    return (n_users, n_movies), (X, y, t), (user_to_index, movie_to_index), (index_to_movieId, index_to_userId)

#################################################################################

def binarize(y, ds): # binarize dataset
    if ds == 'movielens':
      y_binary = deepcopy(y)
      y_binary[y > -1] = 1
    if ds == 'amazon':
      y_binary = deepcopy(y)
      y_binary[y < 4] = 0
      y_binary[y >= 4] = 1
    if ds == 'yahoo':
      y_binary = deepcopy(y)
      y_binary[y < 85] = 0
      y_binary[y == 255] = 0
      y_binary[y >= 85] = 1
    return y_binary

#################################################################################

class ReviewsIterator:

    def __init__(self, X, y, batch_size=32, shuffle=True):
        X, y = np.asarray(X), np.asarray(y)

        if shuffle:
            index = np.random.permutation(X.shape[0])
            X, y = X[index], y[index]

        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n_batches = int(math.ceil(X.shape[0] // batch_size))
        self._current = 0

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def next(self):
        if self._current >= self.n_batches:
            raise StopIteration()
        k = self._current
        self._current += 1
        bs = self.batch_size
        return self.X[k * bs:(k + 1) * bs], self.y[k * bs:(k + 1) * bs]

#################################################################################

def batches(X, y, bs=32, shuffle=True):
    for xb, yb in ReviewsIterator(X, y, bs, shuffle):
        xb = torch.LongTensor(xb)
        yb = torch.FloatTensor(yb)
        yield xb, yb.view(-1, 1)

#################################################################################

def create_user_movie_matrix(X, y, n, m):
    ratings_mat = np.zeros([n, m])
    for i, sample in enumerate(X.values):
        user_idx = sample[0]
        movie_idx = sample[1]
        ratings_mat[user_idx][movie_idx] = y.values[i]
    return ratings_mat

#################################################################################

def filter_dataset(ratings):
    
    ##### filtering users with less then 2 interactions #####
    print('Filtering Data')
    
    (n, m), (X, y_binary,
             t), (user_to_index, movie_to_index), (index_to_movieId,
                                                   index_to_userId) = create_dataset(ratings)

    ratings_mat_full = create_user_movie_matrix(X, y_binary, n, m)
    pos_samples_idx_mat = np.argwhere(ratings_mat_full == 1).tolist()

    pos_samples_idx_dict = {}
    for i in range(n):
        pos_samples_idx_dict[i] = []
    for pos_sample_id in pos_samples_idx_mat:
        pos_samples_idx_dict[pos_sample_id[0]].append(pos_sample_id[1])

    userIds_to_delete = []
    for idx in pos_samples_idx_dict:
        if len(pos_samples_idx_dict[idx]) < 2:
            userId = index_to_userId[idx]
            userIds_to_delete.append(userId)

    ratings.set_index('userId', inplace=True)
    ratings.drop(index=userIds_to_delete, inplace=True)
    ratings.reset_index(inplace=True)

    ##### filtering movies with less then 2 interactions #####
    (n, m), (X, y_binary, t), (user_to_index, movie_to_index), (index_to_movieId, index_to_userId) = create_dataset(
        ratings)

    ratings_mat_full = create_user_movie_matrix(X, y_binary, n, m)
    pos_samples_idx_mat = np.argwhere(ratings_mat_full == 1).tolist()

    pos_samples_idx_dict = {}
    for i in range(m):
        pos_samples_idx_dict[i] = []
    for pos_sample_id in pos_samples_idx_mat:
        pos_samples_idx_dict[pos_sample_id[1]].append(pos_sample_id[0])

    movieIds_to_delete = []
    for idx in pos_samples_idx_dict:
        if len(pos_samples_idx_dict[idx]) < 2:
            movieId = index_to_movieId[idx]
            movieIds_to_delete.append(movieId)

    ratings.set_index('movieId', inplace=True)
    ratings.drop(index=movieIds_to_delete, inplace=True)
    ratings.reset_index(inplace=True)
    ratings.set_index('userId', inplace=True)
    ratings.reset_index(inplace=True)
    return ratings

#################################################################################

def prep_batches(datasets, bs):
    batches_dict = {'train': {'x_batches': [], 'y_batches': []}}
    for batch in batches(*datasets['train'], shuffle=True, bs=bs):
        x_batch, y_batch = [b.to(device) for b in batch]
        x_batch = x_batch.cpu().numpy()
        y_batch = y_batch.cpu().numpy()
        batches_dict['train']['x_batches'].append(x_batch)
        batches_dict['train']['y_batches'].append(y_batch)

    print("# of training batches: ", len(batches_dict['train']['x_batches']))
    return batches_dict

#################################################################################

def create_test_dict(ratings):
    (n, m), (X, y_binary, t), (user_to_index, movie_to_index), (index_to_movieId, index_to_userId) = create_dataset(
        ratings)
    
    ratings_mat_full = create_user_movie_matrix(X, y_binary, n, m)
    pos_samples_idx_mat = np.argwhere(ratings_mat_full == 1).tolist()
    
    pos_samples_idx_dict = {}
    for i in range(n):
        pos_samples_idx_dict[i] = []
    for pos_sample_idx in pos_samples_idx_mat:
        pos_samples_idx_dict[pos_sample_idx[0]].append(pos_sample_idx[1])

    df = pd.DataFrame(index=[ratings.userId.to_list(), ratings.movieId.to_list()],
                      columns=['rating', 'timestamp'])
    df.index.names = ['userId', 'movieId']
    df.rating = ratings.rating.values
    df.timestamp = ratings.timestamp.values

    '''for every user idx, pick a movie idx, add as key:value pair to test dict
    use multiindexing on the ratings dataframe, then drop the relevent rows'''
    test_dict = {}
    ratingIds_to_delete = []
    random.seed(RND_SEED)
    
    for user_idx in pos_samples_idx_dict:
        userId = index_to_userId[user_idx]
        movie_idxs = random.sample(pos_samples_idx_dict[user_idx], k = 1)
        test_dict[userId] = []
        movieId = index_to_movieId[movie_idxs[0]]
        test_dict[userId].append(movieId)
        ratingIds_to_delete.append((userId, movieId))
    
    df.drop(index=ratingIds_to_delete, inplace=True)
    df.reset_index(level=[0, 1], inplace=True)
    
    print("Amount of training samples: ", df.shape[0])
    return df, test_dict

#################################################################################

def prep_trial(ds):
  
    movies = None
          
    if ds == 'yahoo_music_data':
        ratings = pd.read_csv('./Datasets/{0}.csv'.format(ds), header=None, names=['userId', 'movieId', 'rating', 'timestamp'],  engine='python')
        ds_ = 'yahoo'
        
    if ds == 'amazon_Video_Games_data':
        ratings = pd.read_csv('./Datasets/{0}.csv'.format(ds), header=None, names=['userId', 'movieId', 'rating', 'timestamp'],  engine='python')
        ds_ = 'amazon'
      
    if ds == 'ml-latest-small' or ds == 'ml-1m':
        archive_url = f'http://files.grouplens.org/datasets/movielens/{ds}.zip'
        print(archive_url)
        download_path = Path.home() / 'data' / 'movielens'
        try_download(archive_url, download_path)
        ratings, movies = read_data(download_path / ds)
        movies.set_index('movieId', inplace=True)
        ds_ = 'movielens'
    
    #binarize ratings
    ratings['rating'] = binarize(ratings['rating'], ds_)
    
    #filter data (users and items)
    for ff in range(3): # sufficient for our datasets
        ratings = filter_dataset(ratings)
    
    #get test items for users
    ratings, test_dict = create_test_dict(ratings)
    
    (n, m), (X, y_binary, t), (userId_to_index, movieId_to_index), \
    (index_to_movieId, index_to_userId) = create_dataset(ratings)
    print(f'{n} users, {m} items')
    
    datasets = {'train': (X, y_binary)}
    dataset_sizes = {'train': len(X)}
    ratings_mat_full = create_user_movie_matrix(X, y_binary, n, m)   
    
    '''
    Create lookup table for randomally select neg and pos samples for each user
    Table will be: {useridx: [item1_idx, item2_idx, item3_idx, ...]} with the idx's being indexes of
    neg/pos movies for that user'''
    
    print('Creating positive and negative examples for all users...')
    pos_samples_idx_mat = np.argwhere(ratings_mat_full > 0).tolist()
    neg_samples_idx_mat = np.argwhere(ratings_mat_full == 0).tolist()
    
    neg_samples_idx_dict = {}
    for i in range(n):
        neg_samples_idx_dict[i] = []
    for neg_sample_id in neg_samples_idx_mat:
        neg_samples_idx_dict[neg_sample_id[0]].append(neg_sample_id[1])
    
    pos_samples_idx_dict = {}
    for i in range(n):
        pos_samples_idx_dict[i] = []
    for pos_sample_id in pos_samples_idx_mat:
        pos_samples_idx_dict[pos_sample_id[0]].append(pos_sample_id[1])
    
    return (pos_samples_idx_dict, neg_samples_idx_dict, n, m,
            datasets, dataset_sizes, test_dict,
            userId_to_index, movieId_to_index, ratings, index_to_movieId, index_to_userId, movies)

#################################################################################

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#################################################################################
        
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

# Custom Loss Function

In [3]:
class CustomLoss(nn.Module):
    
    def __init__(self, lamda_pos = 0.05, lamda_neg = 0.01, alpha = 0.5):
        super(CustomLoss, self).__init__()
        self.alpha = alpha
        self.lamda_pos = lamda_pos
        self.lamda_neg = lamda_neg
        self.loss_function = torch.nn.CrossEntropyLoss()

    def forward(self, model, users, items, prediction_input, labels, personas_scores_input):
            
        # The loss function without the entropy term over the personas vectors 
        if personas_scores_input is None:
            CE_loss = self.loss_function(prediction_input, labels)
            return CE_loss, CE_loss, torch.Tensor([0]), torch.Tensor([0])
        
        # The loss function for the AMP-CF model 
        CE_loss = self.loss_function(prediction_input, labels)
        personas_scores = personas_scores_input.clone()
        entropy_pos = torch.sum(Categorical(probs = personas_scores[:,:1,:]).entropy())
        entropy_negs = torch.sum(torch.sum(Categorical(probs = personas_scores[:,1:,:]).entropy(), dim = -1) \
                                  / (personas_scores.shape[1] - 1))
        total_loss = (self.alpha)*( CE_loss ) + (1-self.alpha)*( self.lamda_pos * entropy_pos - self.lamda_neg * entropy_negs )
        return (total_loss, CE_loss, entropy_pos, entropy_negs)

# Collaborative Filtering Models

In [4]:
class Collaborative_Filtering(torch.nn.Module):
    
    #### An object that contains all the models in the article (both initialization and forward) ####
    
    def __init__(self, n_users, n_items, n_factors, n_personas, num_samples, bs, pos_samples_idx_dict):
        super().__init__()
        
        self.n_personas = n_personas
        self.device = device
        
        # AMPCF
        if n_personas > 1 and n_personas < 100:
            self.emb_dimension = n_factors
            self.user_factors = torch.nn.Parameter(torch.randn(n_users, n_personas, n_factors))
            self.item_factors = torch.nn.Parameter(torch.randn(n_items, n_factors))

            torch.nn.init.xavier_normal_(self.user_factors)
            torch.nn.init.xavier_normal_(self.item_factors)
        
        # MF
        if n_personas == 1:
            self.emb_dimension = n_factors
            self.user_factors = torch.nn.Parameter(torch.randn(n_users, n_personas, n_factors))
            self.item_factors = torch.nn.Parameter(torch.randn(n_items, n_factors))
            self.users_bias = torch.nn.Parameter(torch.randn(n_users))
            self.items_bias = torch.nn.Parameter(torch.randn(n_items))

            torch.nn.init.xavier_normal_(self.user_factors)
            torch.nn.init.xavier_normal_(self.item_factors)
        
        # SpectralCF
        if n_personas == 444:
            self.n_factors = n_factors
            self.user_factors = torch.nn.Parameter(torch.randn(n_users,n_factors))
            self.item_factors = torch.nn.Parameter(torch.randn(n_items,n_factors))
            torch.nn.init.xavier_normal_(self.user_factors)
            torch.nn.init.xavier_normal_(self.item_factors)

            interaction_matrix = torch.zeros([n_users,n_items])
            for user_idx in range(n_users):
                ind = pos_samples_idx_dict[user_idx]
                interaction_matrix[user_idx][ind] = 1
                
            B1 = torch.cat((torch.zeros(n_users,n_users),interaction_matrix),1)
            B2 = torch.cat((torch.transpose(interaction_matrix,0,1), torch.zeros(n_items,n_items)),1)
            A = torch.cat((B1,B2),0)
            D = torch.diag(torch.sum(A,1))
            L = torch.eye(n_items+n_users)-torch.matmul(torch.inverse(D),A)
            
            G, U = torch.eig(L, eigenvectors=True) 
            G = torch.diag(G[:,0])
            
            Q = torch.matmul(U,torch.transpose(U,0,1))+torch.matmul(torch.matmul(U,G),torch.transpose(U,0,1))
            self.Q = Q.requires_grad_(False).to(device)
            
            self.theta0 = torch.nn.Parameter(torch.randn(n_factors,n_factors))
            self.theta1 = torch.nn.Parameter(torch.randn(n_factors,n_factors))
            self.theta2 = torch.nn.Parameter(torch.randn(n_factors,n_factors))
            torch.nn.init.xavier_normal_(self.theta0)
            torch.nn.init.xavier_normal_(self.theta1)
            torch.nn.init.xavier_normal_(self.theta2)
            
            self.sigmoid = torch.nn.Sigmoid()
        
        #JCA
        if n_personas == 555:
            
            self.n_factors = n_factors
            self.user_factors = torch.nn.Parameter(torch.randn(n_users, n_factors))
            self.item_factors = torch.nn.Parameter(torch.randn(n_items, n_factors))

            interaction_matrix = torch.zeros([n_users,n_items], requires_grad=False)
            for user_idx in range(n_users):
                ind = pos_samples_idx_dict[user_idx]
                interaction_matrix[user_idx][ind] = 1
                
            self.interaction_matrix = interaction_matrix.to(device)
            
            torch.nn.init.xavier_normal_(self.user_factors)
            torch.nn.init.xavier_normal_(self.item_factors)
            
            self.UW = torch.nn.Parameter(torch.randn(n_factors,n_items))
            self.IW = torch.nn.Parameter(torch.randn(n_factors,n_users))
            self.Ub1 = torch.nn.Parameter(torch.randn(1, n_factors))
            self.Ub2 = torch.nn.Parameter(torch.randn(1,n_items))
            self.Ib1 = torch.nn.Parameter(torch.randn(1, n_factors))
            self.Ib2 = torch.nn.Parameter(torch.randn(1,n_users))
            self.f = torch.nn.Parameter(torch.randn(n_items,1))
            
            torch.nn.init.xavier_normal_(self.UW)
            torch.nn.init.xavier_normal_(self.IW)
            torch.nn.init.xavier_normal_(self.Ub1)
            torch.nn.init.xavier_normal_(self.Ub2)
            torch.nn.init.xavier_normal_(self.Ib1)
            torch.nn.init.xavier_normal_(self.Ib2)
            torch.nn.init.xavier_normal_(self.f)
            
            self.sigmoid = torch.nn.Sigmoid()
        
        #CDAE
        if n_personas == 666:
            
            self.n_factors = n_factors
            self.user_factors = torch.nn.Parameter(torch.randn(n_users, n_factors))
            self.item_factors = torch.nn.Parameter(torch.randn(n_items, n_factors))

            interaction_matrix = torch.zeros([n_users,n_items], requires_grad=False)
            for user_idx in range(n_users):
                ind = pos_samples_idx_dict[user_idx]
                interaction_matrix[user_idx][ind] = 1
                
            self.interaction_matrix = interaction_matrix.to(device)
            
            torch.nn.init.xavier_normal_(self.user_factors)
            torch.nn.init.xavier_normal_(self.item_factors)
            
            self.b = torch.nn.Parameter(torch.randn(1,n_factors))
            self.outlayer = torch.nn.Linear(in_features=int(n_factors), out_features=int(n_items))
            torch.nn.init.xavier_normal_(self.b)
            torch.nn.init.xavier_normal_(self.outlayer.weight)
            
            self.sigmoid = torch.nn.Sigmoid()
            self.dropout = torch.nn.Dropout(p=0.6)
        
        #ConvNCF    
        if n_personas == 777:
            self.user_count = n_users
            self.item_count = n_items
            self.num_samples = num_samples
            self.embedding_size = 64
            self.user_factors = torch.nn.Parameter(torch.randn(n_users, n_factors))
            self.item_factors = torch.nn.Parameter(torch.randn(n_items, n_factors))

            torch.nn.init.xavier_normal_(self.user_factors)
            torch.nn.init.xavier_normal_(self.item_factors)

            # cnn setting
            self.channel_size = 32
            self.kernel_size = 2
            self.strides = 2
            self.cnn = nn.Sequential(
                # batch_size * 1 * 64 * 64
                nn.Conv2d(1, self.channel_size, self.kernel_size, stride=self.strides),
                nn.ReLU(),
                # batch_size * 32 * 32 * 32
                nn.Conv2d(self.channel_size, self.channel_size, self.kernel_size, stride=self.strides),
                nn.ReLU(),
                # batch_size * 32 * 16 * 16
                nn.Conv2d(self.channel_size, self.channel_size, self.kernel_size, stride=self.strides),
                nn.ReLU(),
                # batch_size * 32 * 8 * 8
                nn.Conv2d(self.channel_size, self.channel_size, self.kernel_size, stride=self.strides),
                nn.ReLU(),
                # batch_size * 32 * 4 * 4
                nn.Conv2d(self.channel_size, self.channel_size, self.kernel_size, stride=self.strides),
                nn.ReLU(),
                # batch_size * 32 * 2 * 2
                nn.Conv2d(self.channel_size, self.channel_size, self.kernel_size, stride=self.strides),
                nn.ReLU(),
                # batch_size * 32 * 1 * 1
            )
            
            self.fc = nn.Linear(32, 1)
            torch.nn.init.xavier_normal_(self.fc.weight)
        
        #NCF
        if n_personas == 888:
            
            self.n_factors = n_factors
            self.user_factors = torch.nn.Parameter(torch.randn(n_users, 5*n_factors)) #0:n_factors->GMF, n_factors:5*n_factors->MLP
            self.item_factors = torch.nn.Parameter(torch.randn(n_items, 5*n_factors)) #0:n_factors->GMF, n_factors:5*n_factors->MLP
            
            torch.nn.init.xavier_normal_(self.user_factors)
            torch.nn.init.xavier_normal_(self.item_factors)
            
            self.layer1 = torch.nn.Linear(in_features=int(8*n_factors), out_features=int(4*n_factors))
            self.layer2 = torch.nn.Linear(in_features=int(4*n_factors), out_features=int(2*n_factors))
            self.layer3 = torch.nn.Linear(in_features=int(2*n_factors), out_features=int(n_factors))
            
            self.affine_output = torch.nn.Linear(in_features=2*n_factors, out_features=1)
            
            torch.nn.init.xavier_normal_(self.layer1.weight)
            torch.nn.init.xavier_normal_(self.layer2.weight)
            torch.nn.init.xavier_normal_(self.layer3.weight)
            torch.nn.init.xavier_normal_(self.affine_output.weight)
            
            self.relu =   torch.nn.ReLU()
            self.sigmoid = torch.nn.Sigmoid()

    def forward(self, user, items, bs, n_personas, n_factors, test_flag = False):
        
        if n_personas == 1:
            '''
            Forward MF
            '''
            return self.forward_MF(user, items, test_flag)
        
        if n_personas > 1 and n_personas < 100:
            '''
            Forward AMPCF
            '''
            return self.forward_AMPCF(user, items, test_flag)
        
        if n_personas == 444:
            '''
            Forward SpectralCF
            '''
            return self.forward_SpectralCF(user, items, test_flag)
        
        if n_personas == 555:
            '''
            Forward JCA
            '''
            return self.forward_JCA(user, items, test_flag)
        
        if n_personas == 666:
            '''
            Forward CDAE
            '''
            return self.forward_CDAE(user, items, test_flag)
        
        if n_personas == 777:
            '''
            Forward ConvNCF
            '''
            return self.forward_ConvNCF(user, items, test_flag)
        
        if n_personas == 888:
            '''
            Forward NeuMF
            '''
            return self.forward_NeuMF(user, items, test_flag)
        
    def forward_AMPCF(self, user, items, test_flag):
        
        if test_flag == False:
            n_personas = self.n_personas                
            u = self.user_factors[user].squeeze()  # 256,4,128
            v = self.item_factors[items] # 256,5,128
            r = torch.einsum('bsf,bpf->bsp', [v, u])
            attentive_scores = F.softmax(r,dim=-1)
            attentive_user = torch.einsum('bpf,bsp->bsf',[u,attentive_scores])
            pred = torch.einsum('bsf,bsf->bs', [attentive_user,v])
            return(pred,attentive_scores)
            
        if test_flag == True:
            u = self.user_factors[user].squeeze()
            v = self.item_factors[items].squeeze()
            r = torch.einsum('pf,bf->bp', [u, v])
            attentive_scores = F.softmax(r,dim=-1)
            attentive_user = torch.einsum('pf,bp->bf',[u,attentive_scores])
            pred = torch.einsum('bf,bf->b', [attentive_user,v])
            return(pred.squeeze(),attentive_scores,r)
    
    def forward_SpectralCF(self, user, items, test_flag = False):
        
        X0 = torch.cat((self.user_factors,self.item_factors),0)
        X1 = torch.sigmoid(torch.matmul(torch.matmul(self.Q,X0),self.theta0))
        X2 = torch.sigmoid(torch.matmul(torch.matmul(self.Q,X1),self.theta1))
        X3 = torch.sigmoid(torch.matmul(torch.matmul(self.Q,X2),self.theta2))
        
        V = torch.cat((X0,X1,X2,X3),1)
        Vu = V[0:len(self.user_factors),:]
        Vi = V[len(self.user_factors):,:]
        
        if test_flag == False:
            score = torch.einsum('bsf,bpf->bs', [Vi[items,:], Vu[user,:]])
        
        if test_flag == True:
            score = torch.matmul(Vi[items.squeeze(),:],Vu[user.squeeze(),:].squeeze())
            
        if test_flag:
            return score.squeeze(),None,None
        return score.squeeze(),None 
    
    def forward_JCA(self,user,items,test_flag = False):
        
        interaction_batch1 = self.interaction_matrix[user.squeeze()]
        interaction_batch2 = torch.transpose(self.interaction_matrix,0,1)[items.squeeze()]
        
        if test_flag == False:
            
            Z1 = self.sigmoid(torch.einsum('bi,ik->bk',[interaction_batch1,self.item_factors]) + self.Ub1)
            score1 = self.sigmoid(torch.matmul(Z1,self.UW)+self.Ub2)
            
            Z2 = self.sigmoid(torch.mul(torch.einsum('bau,uk->bak',[interaction_batch2,self.user_factors]) + self.Ib1, self.f[items.squeeze()]))
            score2 = self.sigmoid(torch.matmul(Z2,self.IW)+self.Ib2)
            
            score = 0.5*(torch.gather(score1, 1, items) + torch.gather(score2, 2, user.expand(score2.size(0),score2.size(1)).unsqueeze(2)).squeeze())
            
        if test_flag == True:
            interaction_batch1 = interaction_batch1.unsqueeze(0)
            interaction_batch2 = interaction_batch2.unsqueeze(0)
            
            Z1 = self.sigmoid(torch.einsum('bi,ik->bk',[interaction_batch1,self.item_factors]) + self.Ub1)
            score1 = self.sigmoid(torch.matmul(Z1,self.UW)+self.Ub2).squeeze()
            
            Z2 = self.sigmoid(torch.mul(torch.einsum('bau,uk->bak',[interaction_batch2,self.user_factors]) + self.Ib1,self.f[items.squeeze()]))
            score2 = self.sigmoid(torch.matmul(Z2,self.IW)+self.Ib2).squeeze()
            
            score = 0.5*(score1[items.squeeze()] + score2[:,user.squeeze()].squeeze())
            
        if test_flag:
            return score.squeeze(),None,None
        return score.squeeze(),None 

    def forward_CDAE(self,user,items,test_flag = False):
        
        interaction_batch = self.interaction_matrix[user.squeeze()].clone()
        
        if test_flag == False:
            interaction_batch = self.dropout(interaction_batch) # drop-out with prob 0.6
            Z = self.sigmoid(torch.matmul(interaction_batch, self.item_factors) + self.user_factors[user.squeeze()] + self.b)
            Y = self.sigmoid(self.outlayer(Z))
            score = torch.gather(Y, 1, items)
            
        if test_flag == True:
            interaction_batch = interaction_batch.unsqueeze(0)
            Z = self.sigmoid(torch.matmul(interaction_batch,self.item_factors) + self.user_factors[user.squeeze()].unsqueeze(0) + self.b)
            Y = self.sigmoid(self.outlayer(Z)).squeeze()
            score = Y[items]
        
        if test_flag:
            return score.squeeze(),None,None
        return score.squeeze(),None 
    
    def forward_NeuMF(self, user, items, test_flag = False):
        
        user_embedding = self.user_factors[user]
        item_embedding = self.item_factors[items]
        
        if test_flag:
            user_embedding = user_embedding.unsqueeze(0)
            item_embedding = item_embedding.unsqueeze(0)
            
        user_embedding_GMF = user_embedding[:,:,0:self.n_factors]
        item_embedding_GMF = item_embedding[:,:,0:self.n_factors]
        user_embedding_MLP = user_embedding[:,:,self.n_factors:5*self.n_factors]
        item_embedding_MLP = item_embedding[:,:,self.n_factors:5*self.n_factors]
            
        element_product = torch.mul(user_embedding_GMF, item_embedding_GMF)
        
        MLP_vec = torch.cat((user_embedding_MLP.repeat(1, item_embedding_MLP.shape[1], 1), item_embedding_MLP), dim=2)
        aft1 = self.relu(self.layer1(MLP_vec))
        aft2 = self.relu(self.layer2(aft1))
        aft3 = self.relu(self.layer3(aft2))
        
        concat_vec = torch.cat((element_product,aft3), dim=2)  
        aft4 = self.sigmoid(self.affine_output(concat_vec))
        
        if test_flag:
            return aft4.squeeze(),None,None
        
        return aft4.squeeze(),None

    def forward_ConvNCF(self, user_ids, item_ids, test_flag = False):

        user_embeddings = self.user_factors[user_ids]
        item_embeddings = self.item_factors[item_ids]
        
        if test_flag:
            user_embeddings = user_embeddings.unsqueeze(0)
            item_embeddings = item_embeddings.unsqueeze(0)
        
        num_samples = item_embeddings.shape[1]  
        interaction_map = torch.einsum('buf,bsg->bsfg',[user_embeddings,item_embeddings])
        interaction_map = interaction_map.view((-1, 1, self.embedding_size, self.embedding_size))    
        feature_map = self.cnn(interaction_map)
        feature_vec = feature_map.view((-1, 32))
        prediction = self.fc(feature_vec)    
        prediction = prediction.view((-1,num_samples))
        
        if test_flag:
            return prediction.squeeze(),None,None
        return prediction, None        
        
        
    def forward_MF(self, user, items, test_flag = False):
        '''
        Baseline model with single user vector (ie (1,128))
        '''
        u = self.user_factors[user].squeeze()  
        v = self.item_factors[items] 
        if test_flag:
            u = u.squeeze()
            v = v.squeeze()
            bias = self.users_bias[user] + self.items_bias[items]
            return torch.einsum('sf,f->s',[v, u]) + bias,None,None
        else:
            bias = self.users_bias[user] + self.items_bias[items]
            return torch.einsum('bsf,bf->bs',[v, u]) + bias,None

# Model

In [5]:
class Model:
    
    '''This object contains the trained model, the variables and the functions as explained in the comments.
       Most variables are initialized during the run or set at the end.
       The user has the option to change the following fields in the object:
       num_neg_samples, num_random_samples, k.'''
    
    def __init__(self, n_factors, n_personas, pos_samples_idx_dict, neg_samples_idx_dict, n, m,
                 datasets, dataset_sizes, test_dict, userId_to_index, 
                 movieId_to_index, ratings, index_to_movieId, 
                 index_to_userId, num_epochs, bs, lr, alpha, lamda_pos, lamda_neg, patience):
        
        self.device = device
        self.n = n  # num of users
        self.m = m  # num of items
        self.n_factors = n_factors  # number of latent dimensions
        self.n_personas = n_personas  # number of personas per user or model indicator
        self.bs = bs  # batch size
        self.patience = patience # number of epochs to stop the training (HR/NDCG saturation)
        self.n_epochs = num_epochs # number of epochs
        self.lr = lr  # learning rate
        self.wd = 0 # weight decay in Adam optimize
        self.alpha = alpha # a hyperparameter in the loss function
        self.lamda_pos = lamda_pos # a hyperparameter in the loss function
        self.lamda_neg = lamda_neg # a hyperparameter in the loss function
        self.num_neg_samples = 4 # number of negative items per positive item 
        self.num_samples = self.num_neg_samples + 1 # negative samples + positive sample
        self.num_random_samples = 99 # number of random items in test (the 100th item is the test item)
        self.k = 10  # HR@K 
        self.losses = [] # loss array
        self.target = torch.LongTensor([0] * self.bs).to(self.device) # loss function labels
        self.loss_func = CustomLoss(alpha = alpha, lamda_pos = lamda_pos, lamda_neg = lamda_neg) # the loss function
        self.hr_arr = [] # saves the HR results
        self.best_hr = 0 # the best HR in hr_arr
        self.mean_hr = 0 # mean HR at the end of the run (with respect to patience)
        self.ndcg_arr = [] # saves the NDCG results
        self.best_ndcg = 0 # the best NDCG in ndcg_arr
        self.mean_ndcg = 0 # mean NDCG at the end of the run (with respect to patience)
        self.userId_to_index = userId_to_index # user ID to index
        self.index_to_userId = index_to_userId # user index to ID
        self.movieId_to_index = movieId_to_index # item ID to inde
        self.index_to_movieId = index_to_movieId # item index to ID
        self.datasets = datasets # the dataset of the model after data processing
        self.dataset_sizes = dataset_sizes # number of train samples
        self.top_movies_idxs_everyone = None # list of top 30 movies (indices) for each user 
        self.batches_dict = {} # batches dictionary
        self.pos_samples_idx_dict = pos_samples_idx_dict # positive samples for each user
        self.neg_samples_idx_dict = neg_samples_idx_dict # negative samples for each user
        self.test_dict = test_dict # dictionary of users and their test items
        self.test_hashtable = self.get_test_hashtable() # the test items for each user
        self.model = Collaborative_Filtering(n, m, n_factors, n_personas, self.num_samples, self.bs, pos_samples_idx_dict).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.wd,
                                          amsgrad=False)  # the optimizer

    def get_test_hashtable(self):
        test_hashtable = {}
        for userId in self.test_dict:
            user_idx = self.userId_to_index[userId]
            test_hashtable[user_idx] = set()
            for movieId in self.test_dict[userId]:
                if movieId not in self.movieId_to_index:
                    continue
                movie_idx = self.movieId_to_index[movieId]
                test_hashtable[user_idx].add(movie_idx)
        return test_hashtable
    
    def hit(self, gt_item, pred_items): #hit ratio
        pred_items = set(pred_items)
        if gt_item in pred_items:
            return 1
        return 0

    def ndcg(self, gt_item, pred_items):
        pred_items_set = set(pred_items)
        if gt_item in pred_items_set:
            index = pred_items.index(gt_item)
            return np.reciprocal(np.log2(index+2))
        return 0
                
    def HR_and_NDCG_at_K(self, K): #calculating the HR and NDCG
        HR = []
        NDCG = []
        for user_idx in self.test_hashtable:
            test_items = list(self.test_hashtable[user_idx])
            num_test_items = len(test_items)
            if num_test_items == 0:
                continue
            user_hr = 0
            user_ndcg = 0
            random.seed(user_idx)
            for item in test_items:
                random_items = random.sample(self.neg_samples_idx_dict[user_idx], k = 99)
                items = [item] + random_items
                user_scores, _, _ = self.model(torch.LongTensor([user_idx]).to(self.device),
                                                     torch.LongTensor(items).to(self.device), 1,
                                                     self.n_personas, self.n_factors, test_flag = True)
                
                _, indices = torch.sort(user_scores,descending=True)
                recommend_items = torch.take(torch.LongTensor(items), indices.cpu()).cpu().numpy().tolist()
                
                user_hr += self.hit(item, recommend_items[0:K])
                user_ndcg += self.ndcg(item, recommend_items[0:K])
                
            HR.append(user_hr / num_test_items)
            NDCG.append(user_ndcg / num_test_items)
        return np.mean(HR), np.mean(NDCG)

    def get_top_k_movies_idxs_low_ram(self): # Gives the top 30 movies to each user
        k = 30
        user_idxs = [i for i in range(self.n)]
        num_users = self.n
        real_top_k = np.zeros(shape = (num_users,k), dtype = int)
        real_top_k_scores = np.zeros(shape = (num_users,k), dtype = float)
        all_movies_idxs = range(self.m)  
        for user_idx in user_idxs:
            scores , _, _ = self.model(torch.LongTensor([user_idx]).to(self.device),
                                                torch.LongTensor(all_movies_idxs).to(self.device), 1,
                                                self.n_personas, self.n_factors, test_flag = True)
            top_k_movies_idxs = (-scores.detach().cpu().numpy()).argsort()
            ki = 0
            for movie_idx in top_k_movies_idxs:
                if ki == k:
                    break
                if int(movie_idx) in set(self.pos_samples_idx_dict[user_idx]):                  
                    continue
                else:
                    real_top_k[user_idx][ki] = int(movie_idx)
                    real_top_k_scores[user_idx][ki] = scores[movie_idx]
                    ki += 1
        self.top_movies_idxs_everyone = real_top_k

    def train_model(self):         
        
        num_epochs_no_improvement = 0
        patience = self.patience
        self.model.train()
        self.batches_dict = prep_batches(self.datasets, self.bs)
        
        for epoch in range(self.n_epochs):
            if num_epochs_no_improvement+1 > patience:   
                self.mean_hr = (sum(self.hr_arr[(-1*patience-1):]) / (patience+1))
                self.mean_ndcg = (sum(self.ndcg_arr[(-1*patience-1):]) / (patience+1))
                self.best_hr = max(self.hr_arr)
                self.best_ndcg = max(self.ndcg_arr)
                break
            
            print("Epoch: {}".format(epoch + 1), end='\n', flush=True)
            stats = {'epoch': epoch + 1, 'total': self.n_epochs}

            phase = 'train'
            training = True
            running_loss_ce = 0.0
            running_loss_ent_pos = 0.0
            running_loss_ent_neg = 0.0
            running_loss = 0.0
            n_batches = 0
            i = 0

            for x_batch in self.batches_dict['train']['x_batches']:
                self.optimizer.zero_grad()
                with torch.set_grad_enabled(training):  # compute gradients only during 'train' phase
                    batch_user_idxs = x_batch.T[0]  # bs = 5 --> [4, 312, 788, 14, 15]
                    batch_item_idxs = []
                    for user_idx in batch_user_idxs:
                        neg_movies_idxs = random.sample(self.neg_samples_idx_dict[user_idx],
                                                        k=self.num_neg_samples)  # k =4 -> [18, 1898, 8900, 31]
                        pos_movie_idx = random.sample(self.pos_samples_idx_dict[user_idx], k=1)  # [8762]
                        item_idxs = pos_movie_idx + neg_movies_idxs  # [8762, 18, 1898, 8900, 31]
                        batch_item_idxs.append(item_idxs)

                    users = torch.LongTensor([batch_user_idxs]).to(self.device)
                    items = torch.LongTensor(batch_item_idxs).to(self.device)
                    
                    predictions, personas_scores = self.model(users.T, items, self.bs, self.n_personas, self.n_factors)
                    loss, CE_loss, entropy_pos, entropy_negs  = self.loss_func(self.model,self.model.user_factors[users].reshape(-1),self.model.item_factors[items].reshape(-1),predictions, self.target, personas_scores) # Shape = [], ie a scalar
                    
                    # ce_loss, entropy_loss
                    # don't update weights and rates when in 'val' phase
                    if training:
                        loss.backward()
                        self.optimizer.step()
                        del users
                        del items
                        del predictions
                        del personas_scores
                        torch.cuda.empty_cache()

                running_loss += loss.item()
                running_loss_ce += CE_loss.item()
                running_loss_ent_pos += entropy_pos.item()
                running_loss_ent_neg += entropy_negs.item()
                
                self.losses.append(loss.item() / self.bs)
                
                epoch_loss = running_loss / self.dataset_sizes[phase]
                epoch_loss_ce = running_loss_ce / self.dataset_sizes[phase]
                epoch_loss_pos = running_loss_ent_pos / self.dataset_sizes[phase]
                epoch_loss_neg = running_loss_ent_neg / self.dataset_sizes[phase]
                
                stats[phase] = epoch_loss            
            print("CE Loss: {:.4f} Entr Pos Loss: {:.4f} Entr Neg Loss: {:.4f}\n".format(
                epoch_loss_ce, epoch_loss_pos, epoch_loss_neg), end='', flush=True)

            '''HR and NDCG @ K Calc'''
            zero_hr, zero_ndcg = self.HR_and_NDCG_at_K(self.k)
            self.hr_arr.append(zero_hr)
            self.ndcg_arr.append(zero_ndcg)
            improved_hr = False
            improved_ndcg = False
            num_epochs_no_improvement += 1 

            print('HR: {:.4f}'.format(zero_hr), end='  ', flush=True)
            print('NDCG: {:.4f}'.format(zero_ndcg), end='  ', flush=True)

            if self.best_hr < zero_hr:
                self.best_hr = zero_hr
                num_epochs_no_improvement = 0
                improved_hr = True
                
            if self.best_ndcg < zero_ndcg:
                self.best_ndcg = zero_ndcg
                num_epochs_no_improvement = 0
                improved_ndcg = True
                    
            if improved_hr:
                print('HR improved', end=' ', flush=True)
            if improved_ndcg:
                print('NDCG improved', end=' ', flush=True)
            print('\n')

##########################################################################################
########################### After Training the models ####################################
##########################################################################################
    
    '''One can get the recommendation list of a user using: model.get_recommended_movies_names(user_idx),
       the list of positive items of a user using: model.get_training_movies_names(user_idx),
       and the test item of each user using: model.get_test_movies_names(user_idx).'''
    
    def get_recommended_movies_names(self, user_idx):
        movieIds_arr = []
        movie_names_arr = []
        userId = self.index_to_userId[user_idx]
        print(f"userId = {userId}, user_idx = {user_idx} in embedding matrix")
        movie_genre_arr = []
        for index,idx in enumerate(self.top_movies_idxs_everyone[user_idx][0:30]): 
            movieId = self.index_to_movieId[idx.item()]
            movieIds_arr.append(movieId)
            movie_name = movies.loc[movieId,:][0]
            movie_genre = movies.loc[movieId,:][1]
            movie_genre_arr.append([movie_name, movie_genre, idx])
        print(tabulate(movie_genre_arr, headers=['Movie Name', 'Genre', 'movieIdx']))

    def get_training_movies_names(self, user_idx):
        movieIds_arr = []
        movie_names_arr = []
        userId = self.index_to_userId[user_idx]
        print(f"userId = {userId}, user_idx = {user_idx} in embedding matrix")
        print('#### Training Movie Names ####')
        movie_genre_arr = []
        for idx in set(self.pos_samples_idx_dict[user_idx]):
            if idx not in self.test_hashtable[user_idx]:
                movieId = self.index_to_movieId[idx]
                movieIds_arr.append(movieId)
                movie_name = movies.loc[movieId,:][0]
                movie_genre = movies.loc[movieId,:][1]
                movie_genre_arr.append([movie_name, movie_genre, idx])
        print(tabulate(movie_genre_arr, headers=['Movie Name', 'Genre', 'movieIdx']))

    def get_test_movies_names(self, user_idx):
        movieIds_arr = []
        movie_names_arr = []
        userId = self.index_to_userId[user_idx]
        print(f"userId = {userId}, user_idx = {user_idx} in embedding matrix")
        movie_genre_arr = []
        for idx in self.test_hashtable[user_idx]:
            movieId = self.index_to_movieId[idx]
            movieIds_arr.append(movieId)
            movie_name = movies.loc[movieId,:][0]
            movie_genre = movies.loc[movieId,:][1]
            movie_genre_arr.append([movie_name, movie_genre, idx])
        print(tabulate(movie_genre_arr, headers=['Movie Name', 'Genre', 'movieIdx']))

# Train Model

In [6]:
#########################################################################################
'''In this section we train the models.
   The user can control the following parameters:
   num_epochs, bs, lr, patience, 
   choose_dataset, comments, parameters_arr.
   The definition of each variable is noted next to it.'''

base_path = os.getcwd()+'/'
num_epochs = 200
bs = 256 #batch size
lr = 0.001 #learning rate
patience = 4 # number of epochs in which the performance does not increase and therefore we stop the training
choose_dataset = 3 # 1:ML100K, 2:ML1M, 3:Yahoo, 4:Amazon
comments = 'test' # the comment added to the model's name at the end of the run.
                  # The object save function: save_obj(models_dict, os.path.join(base_path + 'models/', f'models_dict_{ds_title}_{comments}'))
                  # For example: models_dict_Movielens 100K_test.pkl.

'''''''''
parameters_arr contains the models' type and hyperparameters:

first field - model type:  MF:1, AMP-X:X (X=[2-99]), SpectralCF:444, JCA:555, CDAE:666, ConvNcf:777, NCF:888
second field - number of latent dimensions, e.g., 8, 16, 32 ...
third field - alpha (a hyperparameter in the loss function)
fourth field - positive lambda (a hyperparameter in the loss function)
fifth field - negative lambda (a hyperparameter in the loss function)
'''''''''

parameters_arr = [(1,8,1,0,0), (3,64,0.5,0.005,0.005)] # In this example we run a MF with 8 latent dimensions
                                                       # and AMP-3 with 64 latent dimensions, alpha=0.5, lambda_p=lambda_n=0.005

#########################################################################################

if choose_dataset == 1:
    ds = 'ml-latest-small'
    ds_title = 'Movielens 100K'
    
if choose_dataset == 2:
    ds = 'ml-1m'
    ds_title = 'Movielens 1M'

if choose_dataset == 3:
    ds = 'yahoo_music_data'
    ds_title = 'Yahoo Music'
    
if choose_dataset == 4:
    ds = 'amazon_Video_Games_data'
    ds_title = 'Amazon Video Games'

    
'''Data Processing'''
print(f'Using device: {device}')
print(f'Training on {ds_title}')
pos_samples_idx_dict, neg_samples_idx_dict, n, m, \
datasets, dataset_sizes, test_dict, userId_to_index, \
movieId_to_index, ratings, index_to_movieId, \
index_to_userId, movies = prep_trial(ds)
print('\n -------Data Prep Finished, Starting Training--------')
print()

models_dict = {}

for model_key in parameters_arr:

    num_personas, num_factors, alpha, lamda_pos, lamda_neg = model_key
    rnd_seed = RND_SEED
    random.seed(RND_SEED)
    torch.manual_seed(RND_SEED)
    np.random.seed(RND_SEED)

    print('model type, #latent dimensions, alpha, positive lambda, negative lambda:', num_personas, num_factors, alpha, lamda_pos, lamda_neg)       
    print()
    
    model = Model(num_factors, num_personas, pos_samples_idx_dict, neg_samples_idx_dict, n, m, datasets,
                dataset_sizes, test_dict, userId_to_index, movieId_to_index, ratings, index_to_movieId,
                index_to_userId, num_epochs = num_epochs, bs = bs, lr = lr, 
                alpha = alpha , lamda_pos = lamda_pos, lamda_neg = lamda_neg, patience = patience)

    model.train_model()
    model.get_top_k_movies_idxs_low_ram()
    models_dict[model_key] = model
    print('\n ######################################################## \n')

save_obj(models_dict, os.path.join(base_path + 'models/', f'models_dict_{ds_title}_{comments}'))

for key in models_dict.keys():
    print(f'Model: {key}')
    print(f'Best HR: {models_dict[key].best_hr:.4f} ||  Best NDCG: {models_dict[key].best_ndcg:.4f}')
    print('\n')
print(f'############## Finished Training Dataset {ds_title} #############')

Using device: cuda
Training on Yahoo Music
Filtering Data
Filtering Data
Filtering Data
Amount of training samples:  398141
14999 users, 5394 items
Creating positive and negative examples for all users...

 -------Data Prep Finished, Starting Training--------

model type, #latent dimensions, alpha, positive lambda, negative lambda: 1 8 1 0 0

# of training batches:  1555
Epoch: 1
CE Loss: 0.0050 Entr Pos Loss: 0.0000 Entr Neg Loss: 0.0000
HR: 0.7201  NDCG: 0.4527  HR improved NDCG improved 

Epoch: 2
CE Loss: 0.0022 Entr Pos Loss: 0.0000 Entr Neg Loss: 0.0000
HR: 0.8348  NDCG: 0.5512  HR improved NDCG improved 

Epoch: 3
CE Loss: 0.0019 Entr Pos Loss: 0.0000 Entr Neg Loss: 0.0000
HR: 0.8507  NDCG: 0.5671  HR improved NDCG improved 

Epoch: 4
CE Loss: 0.0018 Entr Pos Loss: 0.0000 Entr Neg Loss: 0.0000
HR: 0.8561  NDCG: 0.5750  HR improved NDCG improved 

Epoch: 5
CE Loss: 0.0017 Entr Pos Loss: 0.0000 Entr Neg Loss: 0.0000
HR: 0.8634  NDCG: 0.5873  HR improved NDCG improved 

Epoch: 6
CE

In [7]:
# In order to get the HR@K and NDCG@K for different K's use: model.HR_and_NDCG_at_K(K)
# The first number will be the HR@K and the second one the NDCG@K.

# Load model

In [8]:
'''In this section, you can load the trained models.
   
   Inputs: 
   dataset_name (one of the datasets given in the comment)
   comment (as in the previous section)
   
   Output:
   models_dict - a dictionary of the loaded models
   '''

dataset_name = 'Movielens 100K' #'Movielens 100K', 'Movielens 1M', 'Yahoo Music', 'Amazon Video Games
comment = 'test'

base_path = os.getcwd()+'/'
models_dict = load_obj(os.path.join(base_path + 'models/', f'models_dict_{dataset_name}_{comment}'))

print('Finish Loading')

Finish Loading
